In [1]:
%pwd

'c:\\Users\\wadhw\\Desktop\\My_Projects\\MediAssist-GenAI-ChatBot\\research'

In [2]:
import os
os.chdir("../")

In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [5]:
extracted_data=load_pdf_file(data='Data/')

In [6]:

#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [7]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5859


In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

In [9]:

#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [10]:

embeddings = download_hugging_face_embeddings()

C:\Users\wadhw\AppData\Local\Temp\ipykernel_33776\2503581564.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\wadhw\miniconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:

query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [13]:

PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GOOGLE_API_KEY=os.environ.get('GOOGLE_API_KEY')

In [14]:

import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [15]:
index_name = "medicalbot"

In [21]:
pip uninstall -y pinecone-plugin-inference


Found existing installation: pinecone-plugin-inference 1.1.0Note: you may need to restart the kernel to use updated packages.

Uninstalling pinecone-plugin-inference-1.1.0:
  Successfully uninstalled pinecone-plugin-inference-1.1.0


In [22]:
pip install -U pinecone

Note: you may need to restart the kernel to use updated packages.


In [16]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [17]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [18]:
docsearch

In [19]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [20]:

retrieved_docs = retriever.invoke("What is Acne?")

In [21]:

retrieved_docs

[Document(metadata={'page': 39.0, 'source': 'Data\\Medical_book (1).pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(metadata={'page': 38.0, 'source': 'Data\\Medical_book (1).pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(metadata={'page': 37.0, 'source': 'Data\\Medical_book (1).pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteria.\nDescription\nAcne vulgaris, the me

In [22]:
pip install -U langchain-google-genai google-generativeai

  Using cached langchain_google_genai-3.0.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_core-1.0.4-py3-none-any.whl.metadata (3.5 kB)
  Using cached google_ai_generativelanguage-0.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached google_api_core-2.28.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached google_auth-2.43.0-py2.py3-none-any.whl.metadata (6.6 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached grpcio_status-1.76.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached cachetools-6.2.1-py3-none-any.whl.metadata (5.5 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached protobuf-6.33.0-cp310-abi3-win_amd64.whl.metadata (593 bytes)
  Using cached langsmith-0.4.42-py3-none-any.whl.metadata (14 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
INFO: pi

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.2.13 requires langchain-core<0.3.0,>=0.2.30, but you have langchain-core 0.3.79 which is incompatible.
langchain 0.2.13 requires langsmith<0.2.0,>=0.1.17, but you have langsmith 0.4.42 which is incompatible.
langchain-community 0.2.12 requires langchain-core<0.3.0,>=0.2.30, but you have langchain-core 0.3.79 which is incompatible.
langchain-community 0.2.12 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.4.42 which is incompatible.
langchain-experimental 0.0.64 requires langchain-core<0.3.0,>=0.2.27, but you have langchain-core 0.3.79 which is incompatible.
langchain-openai 0.1.25 requires langchain-core<0.3.0,>=0.2.40, but you have langchain-core 0.3.79 which is incompatible.
langchain-pinecone 0.1.3 requires langchain-core<0.3,>=0.1.52, but you have langchain-core 0.3.79 which is incompati

In [27]:
!pip install -U \
  langchain==0.2.13 \
  langchain-core==0.2.30 \
  langchain-community==0.2.11 \
  langchain-google-genai==1.0.5 \
  google-generativeai==0.5.2 \
  pydantic==1.10.15

  Using cached langchain_core-0.2.30-py3-none-any.whl.metadata (6.2 kB)
  Using cached langchain_community-0.2.11-py3-none-any.whl.metadata (2.7 kB)
  Using cached langchain_google_genai-1.0.5-py3-none-any.whl.metadata (3.8 kB)
  Using cached google_generativeai-0.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached pydantic-1.10.15-cp310-cp310-win_amd64.whl.metadata (152 kB)
  Using cached langsmith-0.1.147-py3-none-any.whl.metadata (14 kB)
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
  Using cached grpcio_status-1.76.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached grpcio_status-1.75.1-py3-none-any.whl.metadata (1.1 kB)
  Using cached grpcio_status-1.75.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached grpcio_status-1.74.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached grpcio_status-1.73.1-py3-none-any.whl.metadata (1.1 kB)
  Using cached grpcio_status-1.73.0-py3-none-any

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.1.25 requires langchain-core<0.3.0,>=0.2.40, but you have langchain-core 0.2.30 which is incompatible.


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",   
    temperature=0.4,
    max_output_tokens=8196
)


In [29]:

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [30]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [31]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [33]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly is a disorder where the abnormal release of a chemical from the pituitary gland causes increased growth in bone and soft tissue after bone growth has stopped. Gigantism is a disorder where the abnormal release of growth hormone from the pituitary gland causes excessive growth of the body's long bones, leading to unusual height. This occurs when the abnormality happens before bone growth stops.


In [34]:

response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

I'm sorry, but the provided text does not contain a definition for "stats." It mentions "blood count" and provides some statistical figures related to sudden unexplained death (SUD) in athletes, but it does not define the general term "stats."
